In [ ]:
import os
import sys

dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [1]:
import io
import time

import paramiko

from utils.config_utils import find_root_dir

from private_keys import GATEWAY_PASSWORD, MYRIAD_PRIVATE_KEY, GITHUB_PRIVATE_KEY

In [2]:
SCRIPT_TO_RUN = 'run_experiment_test.sh'  # 'run_experiment_1.sh', 'run_experiment_2.sh', etc.

In [34]:
def sftp_upload_dir(sftp, localdir, remotedir):
    os.chdir(os.path.split(localdir)[0])
    parent = os.path.split(localdir)[1]
    for walker in os.walk(parent):
        try:
            sftp.mkdir(os.path.join(remotedir, walker[0]))
        except Exception as e:
            print("Exception:", e)
        for file in walker[2]:
            sftp.put(os.path.join(walker[0], file), os.path.join(remotedir, walker[0], file))


## Connecting to Myriad

In [3]:
# Private key
k = MYRIAD_PRIVATE_KEY.strip()
padding = len(k) % 4
if padding > 0:
    k += "=" * (4 - padding)
private_key_file = io.StringIO(k)
pkey = paramiko.RSAKey.from_private_key(private_key_file)

# Establish SSH connection to the first server (gateway)
sshgat = paramiko.SSHClient()
sshgat.set_missing_host_key_policy(paramiko.AutoAddPolicy())
sshgat.connect('ssh-gateway.ucl.ac.uk', username='ucabscm', password=GATEWAY_PASSWORD)

# Use this connection to establish a second SSH connection to the second server (myriad)
# Paramiko Transport is used here for nested ssh
transport = sshgat.get_transport()
dest_addr = ('myriad.rc.ucl.ac.uk', 22) # IP and port
local_addr = ('ssh-gateway.ucl.ac.uk', 22) # IP and port
jump = transport.open_channel("direct-tcpip", dest_addr, local_addr)

myriad = paramiko.SSHClient()
myriad.set_missing_host_key_policy(paramiko.AutoAddPolicy())
myriad.connect('myriad.rc.ucl.ac.uk', username='ucabscm', sock=jump, pkey=pkey)

## Set up environment, code, files

In [5]:
ARDT_DIR = find_root_dir()

In [ ]:
# Run commands on the second server (myriad)
myriad.exec_command(f'rm -rf /cs/student/msc/dsml/2022/amarques/action-robust-decision-transformer')
myriad.exec_command(f'git clone https://{GITHUB_PRIVATE_KEY}@github.com/afonsosamarques/action-robust-decision-transformer.git')

myriad.exec_command('source /usr/local/cuda/CUDA_VISIBILITY.csh')
myriad.exec_command('source /opt/Python/Python-3.10.1_Setup.csh')

myriad.exec_command('mkdir ardt-env')
myriad.exec_command('python3.10 -m venv ./ardt-env/ardt')
myriad.exec_command('source /cs/student/msc/dsml/2022/amarques/ardt-env/ardt/bin/activate.csh')

myriad.exec_command('pip3 install --upgrade pip')
myriad.exec_command('pip3 install -r /cs/student/msc/dsml/2022/amarques/action-robust-decision-transformer/requirements.txt')

myriad.exec_command('mkdir eval-outputs')
myriad.exec_command('mkdir eval-outputs-pipeline')
myriad.exec_command('mkdir eval-outputs-test')
myriad.exec_command('mkdir agents')
myriad.exec_command('mkdir agents-pipeline')
myriad.exec_command('mkdir agents-test')
myriad.exec_command('mkdir wandb')
myriad.exec_command('mkdir wandb-json')

In [12]:
sftp = myriad.open_sftp()

sftp.put(f'{ARDT_DIR}/access_tokens.py', "/home/ucabscm/access_tokens.py")
myriad.exec_command('mv ./access_tokens.py /cs/student/msc/dsml/2022/amarques/action-robust-decision-transformer/ardt')

sftp.put(f'{ARDT_DIR}/all-notebooks/clusters-nbs/myriad-run-scripts/{SCRIPT_TO_RUN}', "/home/ucabscm/{SCRIPT_TO_RUN}")
myriad.exec_command(f'chmod +x {SCRIPT_TO_RUN}')

sftp_upload_dir(sftp, f'{ARDT_DIR}/datasets', "/home/ucabscm/action-robust-decision-transformer/ardt/datasets")

## Run script and close connections

In [ ]:
myriad.exec_command(f'qsub {SCRIPT_TO_RUN}')

In [ ]:
time.sleep(60)
sftp.close()
myriad.close()
sshgat.close()